In [2]:
import numpy as np
import pandas as pd
import pycomlink as pycml
import xarray as xr

# CML processing

In [3]:
def RADclassifier(cmls, rad):
    """wet/dry classfication with radar

    radar frequency 15m
    cml frequency 1m
    """
    TimeSeq = pd.date_range(cmls.time.values[0], cmls.time.values[-1], freq="min")
    cmls = cmls.reindex({"time": TimeSeq})
    cmls["wet"] = (
        ("cml_id", "sublink_id", "time"),
        np.zeros(cmls.rsl.shape, dtype="bool"),
    )
    latitudes = np.array(
        [
            rad.lat.values,
        ]
        * len(rad.lon.values)
    ).transpose()
    longitudes = np.array(
        [
            rad.lon.values,
        ]
        * len(rad.lat.values)
    )
    da_intersect_weights = (
        pycml.spatial.grid_intersection.calc_sparse_intersect_weights_for_several_cmls(
            x1_line=cmls.site_0_lon.values,
            y1_line=cmls.site_0_lat.values,
            x2_line=cmls.site_1_lon.values,
            y2_line=cmls.site_1_lat.values,
            cml_id=cmls.cml_id.values,
            x_grid=longitudes,
            y_grid=latitudes,
            grid_point_location="center",
        )
    )
    radar_along_cmls = (
        pycml.spatial.grid_intersection.get_grid_time_series_at_intersections(
            grid_data=rad.rainfall_amount,
            intersect_weights=da_intersect_weights,
        )
    )
    for ID in cmls.cml_id.values:
        radAlong = radar_along_cmls.sel(cml_id=ID)
        dry_times = radAlong.time.where(radAlong == 0, drop=True).values
        mask = np.zeros(len(cmls.time), dtype="bool")
        if len(dry_times) > 0:
            dry_intervals = np.array(
                [(t - np.timedelta64(15, "m"), t) for t in dry_times]
            )
            mask = np.any(
                [
                    (cmls.time.values > start) & (cmls.time.values <= end)
                    for start, end in dry_intervals
                ],
                axis=0,
            )
        cmls["wet"].loc[{"cml_id": ID, "sublink_id": "channel1"}] = ~mask
        cmls["wet"].loc[{"cml_id": ID, "sublink_id": "channel2"}] = ~mask
    return cmls

In [4]:
def BASELINE(cmls, n_average_last_dry):
    """Compute baseline (median of dry signal)"""
    cmls["baseline"] = (
        ("cml_id", "sublink_id", "time"),
        np.zeros(cmls.rsl.shape, dtype=np.float64),
    )
    print("Baseline computation")
    for ID in cmls.cml_id.values:
        for sub in cmls.sublink_id.values:
            trsl = cmls.tl.sel(cml_id=ID, sublink_id=sub).values
            wet = cmls.wet.sel(cml_id=ID, sublink_id=sub).values
            baseline = np.zeros_like(trsl, dtype=np.float64)
            baseline[0:n_average_last_dry] = trsl[0:n_average_last_dry]
            for i in range(n_average_last_dry, len(trsl)):
                if np.isnan(wet[i]):
                    baseline[i] = np.nan
                elif wet[i] & ~wet[i - 1]:
                    baseline[i] = np.nanmedian(baseline[(i - n_average_last_dry) : i])
                elif wet[i] & wet[i - 1]:
                    baseline[i] = baseline[i - 1]
                else:
                    baseline[i] = trsl[i]
            cmls["baseline"].loc[{"cml_id": ID, "sublink_id": sub}] = baseline
    return cmls

In [5]:
def CMLprocessing(cmls):
    """Process all CMLs"""
    cmls.coords["polarization"] = cmls.polarization.fillna("vertical")
    cmls.coords["frequency"] = cmls.frequency.fillna(25000.0)
    cmls["tl"] = cmls.tsl - cmls.rsl
    cmls = BASELINE(cmls, n_average_last_dry=60)
    cmls["A"] = cmls.tl - cmls.baseline
    cmls["A"] = cmls.A.where(cmls.A >= 0, 0)
    cmls["waa"] = pycml.processing.wet_antenna.waa_leijnse_2008_from_A_obs(
        A_obs=cmls.A,
        f_Hz=cmls.frequency * 1e6,
        pol=cmls.polarization,
        L_km=cmls.length,
        gamma=(2.06e-05) / 1.4,
        l_antenna=0.0043,
        delta=0.24 * 1.5,
    )
    cmls["A_rain"] = cmls.tl - cmls.baseline - cmls.waa
    cmls["A_rain"].values[cmls.A_rain < 0] = 0
    cmls["R"] = pycml.processing.k_R_relation.calc_R_from_A(
        A=cmls.A_rain,
        L_km=cmls.length.astype(float) / 1000,
        f_GHz=cmls.frequency / 1000,  # convert to GHz
        pol=cmls.polarization,
    )
    cmls["R"] = cmls["R"] * (1 / 60)  # rain rate - rain depth conversion
    return cmls.drop_vars({"rsl", "tsl", "wet", "tl", "baseline", "A", "A_rain", "waa"})

# Load data and execute functions

In [6]:
# open data
cmls = xr.open_dataset("CML_20220917_20220918.nc")
rad = xr.open_dataset("RAD_20220917_20220918.nc")
# processing
cmls = RADclassifier(cmls, rad)
cmls = CMLprocessing(cmls)

Baseline computation


/tmp/ecovi/ipykernel_272416/3882095209.py:17: RuntimeWarning: All-NaN slice encountered
  baseline[i] = np.nanmedian(baseline[(i - n_average_last_dry) : i])


# Aggregate precipitation over 15M, Resampling and Slice

In [7]:
cmls = cmls.rolling(time=15, center=False).sum(skipna=False)
TimeSeq = pd.date_range(cmls.time.values[0], cmls.time.values[-1], freq="15min")
cmls = cmls.reindex({"time": TimeSeq})
cmls = cmls.sel(time=slice("2022-09-17T08:00:00", "2022-09-17T10:30"))

# Saving output

In [8]:
cmls = cmls.fillna(-9999.0)
cmls.time.encoding["units"] = "seconds since 1970-01-01 00:00:00"
cmls.time.encoding["calendar"] = "gregorian"
compVar = {"_FillValue": None, "missing_value": -9999.0}
encoding = {var: compVar for var in cmls.data_vars}
cmls.site_0_lat.encoding["_FillValue"] = None
cmls.site_0_lon.encoding["_FillValue"] = None
cmls.site_0_elev.encoding["_FillValue"] = None
cmls.site_1_lat.encoding["_FillValue"] = None
cmls.site_1_lon.encoding["_FillValue"] = None
cmls.site_1_elev.encoding["_FillValue"] = None
cmls.length.encoding["_FillValue"] = None
cmls.frequency.encoding["_FillValue"] = None
cmls.polarization.encoding["_FillValue"] = None
cmls.polarization.encoding["missing_value"] = "NA"
cmls.to_netcdf("openrainer_CML.nc", encoding=encoding)